<a href="https://colab.research.google.com/github/Ino54/MA_GreenAI-Practical-Experiments/blob/main/deepseek_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Requirements ---
%%writefile requirements.txt
transformers>=4.44
accelerate>=0.33
bitsandbytes
datasets>=2.20
evaluate>=0.4
sacrebleu>=2.4
codecarbon>=2.5,<3
psutil
pynvml>=12,<13
numpy==2.0.2
pandas==2.2.2
huggingface_hub

Writing requirements.txt


In [ ]:
!pip install -q -U -r requirements.txt --no-warn-conflicts
!pip uninstall -y -q google-genai firebase-admin || true
!pip show fief-client || echo "fief-client nicht installiert"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.6/517.6 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.2/291.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# --- Google Drive mounten ---
from google.colab import drive
drive.mount('/content/drive')

# --- Projektordner setzen ---
import os, pathlib, re, platform
project_path = "/content/drive/MyDrive/LLM-Effizienz/4_2_Baseline"
pathlib.Path(project_path).mkdir(parents=True, exist_ok=True)
os.chdir(project_path)
print("Arbeitsordner:", os.getcwd())

# --- Hugging Face Login via Colab-Secret ---
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get("HF_TOKEN")
if hf_token:
    login(hf_token)
    print("Hugging Face Login erfolgreich!")
else:
    print("WARNUNG: Kein HF_TOKEN.")

Mounted at /content/drive
Arbeitsordner: /content/drive/MyDrive/LLM-Effizienz/4_2_Baseline
Hugging Face Login erfolgreich!


In [ ]:
# --- Imports & Setup ---
import warnings; warnings.filterwarnings("ignore")
import time, math, gc, inspect
from dataclasses import dataclass, asdict
from contextlib import nullcontext
from typing import Optional
from types import SimpleNamespace

import psutil, numpy as np, pandas as pd, torch
from datasets import load_dataset
import evaluate
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
                          set_seed, GenerationConfig)
from codecarbon import EmissionsTracker, __version__ as cc_ver

print("CodeCarbon-Version:", cc_ver)

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    gpu_name = torch.cuda.get_device_name(0)
    vram_total_gb = torch.cuda.get_device_properties(0).total_memory/(1024**3)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.set_grad_enabled(False)
else:
    gpu_name = "CPU"; vram_total_gb = 0.0

print(f"Device: {device} | GPU: {gpu_name} | VRAM: {vram_total_gb:.1f} GB | Torch {torch.__version__} | Python {platform.python_version()}")

CodeCarbon-Version: 2.8.4
Device: cuda | GPU: NVIDIA A100-SXM4-40GB | VRAM: 39.6 GB | Torch 2.8.0+cu126 | Python 3.12.11


In [ ]:
# ========== Standort-Konfiguration (Berlin) + CodeCarbon-Helper ==========
USE_GCP_REGION = True
COUNTRY_ISO_CODE = "DEU"
CLOUD_PROVIDER   = "google"
CLOUD_REGION     = "europe-west10"

import os as _os, time as _time

def tracker_kwargs_base():
    base = dict(log_level="error", output_dir=".")
    try:
        sig = inspect.signature(EmissionsTracker.__init__)
        if "measure_power_secs" in sig.parameters: base["measure_power_secs"] = 1
        if "tracking_mode" in sig.parameters:      base["tracking_mode"] = "process"
        if USE_GCP_REGION:
            if "cloud_provider" in sig.parameters: base["cloud_provider"] = CLOUD_PROVIDER
            if "cloud_region"   in sig.parameters: base["cloud_region"]   = CLOUD_REGION
            if "country_iso_code" in sig.parameters: base["country_iso_code"] = COUNTRY_ISO_CODE
        else:
            if "country_iso_code" in sig.parameters: base["country_iso_code"] = COUNTRY_ISO_CODE
            if "cloud_provider" in sig.parameters:   base["cloud_provider"] = None
            if "cloud_region"   in sig.parameters:   base["cloud_region"]   = None
    except Exception:
        pass
    return base

def make_tracker_named(project_name: str, output_file: str):
    cache_dir = f"/content/.codecarbon_cache_{project_name}_{int(_time.time())}"
    _os.environ["CODECARBON_CACHE_DIR"] = cache_dir
    for d in (_os.path.expanduser("~/.codecarbon"), "/content/.codecarbon"):
        lock_file = _os.path.join(d, "codecarbon.lock")
        if os.path.exists(lock_file):
            try: os.remove(lock_file)
            except: pass
    return EmissionsTracker(project_name=project_name, output_file=output_file, **tracker_kwargs_base())

def safe_start(tracker):
    try:
        tracker.start(); return True
    except Exception as e:
        print(f"[CodeCarbon] Start fehlgeschlagen: {e} → Fallback 0/0."); return False

def safe_stop(tracker, started: bool):
    if not started: return SimpleNamespace(energy_consumed=0.0, emissions=0.0)
    try:
        return tracker.stop()
    except Exception as e:
        print(f"[CodeCarbon] Stop fehlgeschlagen: {e} → Fallback 0/0.")
        return SimpleNamespace(energy_consumed=0.0, emissions=0.0)

def unpack_emissions(em):
    if em is None: return 0.0, 0.0
    if hasattr(em,"energy_consumed") and hasattr(em,"emissions"):
        try: return float(em.energy_consumed), float(em.emissions)
        except: pass
    if isinstance(em, dict):
        e = em.get("energy_consumed", 0.0); c = em.get("emissions", em.get("emissions_kg", 0.0))
        try: return float(e), float(c)
        except: return 0.0, 0.0
    try: return 0.0, float(em)
    except: return 0.0, 0.0

def read_energy_from_log(path: str) -> float:
    try:
        if not os.path.exists(path): return 0.0
        df = pd.read_csv(path)
        for c in ["energy_consumed","energy_consumed_kwh","energy_consumed (kWh)","energy (kWh)"]:
            if c in df.columns: return float(df[c].iloc[-1])
        for c in df.columns:
            n = c.lower()
            if "energy" in n and "kwh" in n: return float(df[c].iloc[-1])
    except Exception:
        pass
    return 0.0

def measure_phase(phase_name: str, fn, log_prefix: str):
    logfile = f"{log_prefix}_{phase_name}.csv"
    tracker = make_tracker_named(project_name=f"{log_prefix}_{phase_name}", output_file=logfile)
    started = safe_start(tracker)
    t0 = time.time(); result = fn(); t1 = time.time()
    em_raw = safe_stop(tracker, started)
    energy_kwh, co2_kg = unpack_emissions(em_raw)
    if not energy_kwh or energy_kwh == 0.0:
        energy_from_csv = read_energy_from_log(logfile)
        if energy_from_csv: energy_kwh = energy_from_csv
    return {"phase": phase_name, "time_s": t1 - t0, "energy_kwh": energy_kwh, "co2_kg": co2_kg}, result

In [ ]:
# --- Evaluation-Config ---
MODEL_ID   = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
ALIAS_LONG = "DeepSeek-R1-Distill-Qwen-1.5B"
ALIAS      = "r1q15b"

EVAL = {
    "max_new_tokens": 32,
    "ppl_dataset":  {"name":"wikitext","config":"wikitext-2-raw-v1","split":"test[:1%]"},
    "bleu_dataset": {"name":"wmt14","config":"de-en","split":"test[:32]"},
}
import re
def parse_subset_count(split_str: str, default=32):
    m = re.search(r":\s*(\d+)\s*\]$", split_str or ""); return int(m.group(1)) if m else default
BLEU_N = parse_subset_count(EVAL["bleu_dataset"]["split"], default=32)

PROMPTS = [
    "Schreibe einen kurzen Absatz über nachhaltige KI.",
    "Erkläre in einfachen Worten, was Quantisierung in neuronalen Netzen ist.",
    "Nenne drei Vorteile von Mixture-of-Experts-Modellen."
]

def autocast_ctx():
    return torch.autocast(device_type="cuda", dtype=torch.float16) if device=="cuda" else nullcontext()

def capture_memory():
    ram = psutil.Process().memory_info().rss
    valloc = torch.cuda.memory_allocated() if device=="cuda" else 0
    vres  = torch.cuda.memory_reserved()  if device=="cuda" else 0
    return ram, valloc, vres

def bytes_to_gb(b): return float(b)/(1024**3)
bleu_metric = evaluate.load("sacrebleu")

@dataclass
class BaselineResult:
    model_id: str
    alias: str
    precision: str
    time_s: float
    energy_kwh: float
    co2_kg: float
    tokens_out: int
    ram_GB: float
    vram_alloc_GB: float
    vram_reserved_GB: float
    ppl: Optional[float] = None
    bleu: Optional[float] = None
    notes: str = ""

In [ ]:
# --- Utility: safe_max_len + stabile GenerationConfig (keine temperature/top_p-Warnung) ---
def safe_max_len(tok, model, fallback=2048, upper=100000):
    cand = getattr(tok, "model_max_length", None)
    if isinstance(cand, int) and 0 < cand < upper: return cand
    cand = getattr(getattr(model, "config", None), "max_position_embeddings", None)
    if isinstance(cand, int) and 0 < cand < upper: return cand
    return fallback

GC_GREEDY = GenerationConfig(
    do_sample=False,
    temperature=None,
    top_p=None,
    top_k=None,
    num_beams=1,
)

In [ ]:
# --- Modell-Lader (erst FP16, bei OOM dann INT8) ---
def load_model(model_id: str):
    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    if tok.pad_token_id is None:
        tok.pad_token = tok.eos_token
    tok.padding_side = "left"

    try:
        if device == "cuda":
            torch.cuda.empty_cache(); gc.collect()
        model = AutoModelForCausalLM.from_pretrained(
            model_id, device_map="auto",
            torch_dtype=torch.float16 if device=="cuda" else torch.float32,
            attn_implementation="sdpa",
        )
        prec = "fp16" if device=="cuda" else "fp32"
        return tok, model, prec
    except RuntimeError as e:
        if "out of memory" not in str(e).lower(): raise
        print(f"[Info] OOM bei FP16 für {model_id}. Fallback auf INT8…")

    if device == "cuda":
        torch.cuda.empty_cache(); gc.collect()
    bnb = BitsAndBytesConfig(load_in_8bit=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_id, device_map="auto", quantization_config=bnb,
        torch_dtype=torch.float16 if device=="cuda" else torch.float32,
        attn_implementation="sdpa",
    )
    return tok, model, "int8"

def warmup(model, tok, max_len):
    with torch.no_grad(), autocast_ctx():
        dummy = tok("Hello", return_tensors="pt", truncation=True, max_length=max_len).to(model.device)
        _ = model.generate(**dummy, max_new_tokens=1, generation_config=GC_GREEDY, pad_token_id=tok.eos_token_id)

In [ ]:
# --- Generation / PPL / BLEU (analog zu BLOOM, aber mit GC_GREEDY) ---
def simple_generate(model, tok, prompts, max_new_tokens=32):
    model.eval(); total_gen_tokens, texts = 0, []
    max_len = safe_max_len(tok, model)
    for p in prompts:
        enc = tok(p, return_tensors="pt", truncation=True, max_length=max_len).to(model.device)
        room = max_len - enc["input_ids"].shape[1]
        cur_new = max(1, min(max_new_tokens, int(room)))
        with torch.no_grad(), autocast_ctx():
            out = model.generate(
                **enc,
                max_new_tokens=cur_new,
                generation_config=GC_GREEDY,
                pad_token_id=tok.eos_token_id
            )
        gen_len = out.shape[1] - enc["input_ids"].shape[1]
        total_gen_tokens += int(gen_len)
        texts.append(tok.decode(out[0], skip_special_tokens=True))
    return texts, total_gen_tokens

def eval_perplexity(model, tok, ds_cfg):
    ds = load_dataset(ds_cfg["name"], ds_cfg["config"], split=ds_cfg["split"])
    max_len = safe_max_len(tok, model); losses = []
    with torch.no_grad():
        for t in ds["text"]:
            if not isinstance(t, str) or len(t.strip()) < 4: continue
            enc = tok(t, return_tensors="pt", truncation=True, max_length=max_len).to(model.device)
            with autocast_ctx(): out = model(enc["input_ids"], labels=enc["input_ids"])
            losses.append(float(out.loss.detach().cpu()))
    return math.exp(np.mean(losses)) if losses else None

def eval_bleu_llm(model, tok, ds_cfg, max_new_tokens=32):
    ds = load_dataset(ds_cfg["name"], ds_cfg["config"], split=ds_cfg["split"])
    max_len = safe_max_len(tok, model); preds, refs = [], []
    with torch.no_grad():
        for ex in ds:
            de, en = ex["translation"]["de"], ex["translation"]["en"]
            prompt = f"Translate to English:\nGerman: {de}\nEnglish:"
            inputs = tok(prompt, return_tensors="pt", truncation=True, max_length=max_len).to(model.device)
            room = max_len - inputs["input_ids"].shape[1]
            cur_new = max(1, min(max_new_tokens, int(room)))
            with autocast_ctx():
                out = model.generate(
                    **inputs,
                    max_new_tokens=cur_new,
                    generation_config=GC_GREEDY,
                    pad_token_id=tok.eos_token_id
                )
            gen = tok.decode(out[0], skip_special_tokens=True)
            seg = gen.split("English:")[-1].strip()
            hyp = seg.split("\n")[0].strip() or gen.strip()
            preds.append(hyp); refs.append([en])
    return float(bleu_metric.compute(predictions=preds, references=refs)["score"])

In [ ]:
# --- Baseline-Lauf (pro Phase) ---
def run_baseline(model_id: str, alias_long: str, alias_short: str):
    tok, model, prec = load_model(model_id)
    max_len = safe_max_len(tok, model)
    warmup(model, tok, max_len)  # Warm-up außerhalb der Messung
    log_prefix = f"deepseek_{alias_short}"

    def _do_gen(): return simple_generate(model, tok, PROMPTS, EVAL["max_new_tokens"])
    gen_metrics, (examples, tokens_out) = measure_phase("gen", _do_gen, log_prefix)

    def _do_ppl(): return eval_perplexity(model, tok, EVAL["ppl_dataset"])
    ppl_metrics, ppl = measure_phase("ppl", _do_ppl, log_prefix)

    if device == "cuda": torch.cuda.empty_cache()
    def _do_bleu(): return eval_bleu_llm(model, tok, EVAL["bleu_dataset"], EVAL["max_new_tokens"])
    bleu_metrics, bleu = measure_phase("bleu", _do_bleu, log_prefix)

    total_time   = gen_metrics["time_s"] + ppl_metrics["time_s"] + bleu_metrics["time_s"]
    total_energy = gen_metrics["energy_kwh"] + ppl_metrics["energy_kwh"] + bleu_metrics["energy_kwh"]
    total_co2    = gen_metrics["co2_kg"] + ppl_metrics["co2_kg"] + bleu_metrics["co2_kg"]

    ram = psutil.Process().memory_info().rss
    valloc = torch.cuda.memory_allocated() if device=="cuda" else 0
    vres  = torch.cuda.memory_reserved()  if device=="cuda" else 0

    per_phase_df = pd.DataFrame([gen_metrics, ppl_metrics, bleu_metrics])
    per_phase_df["alias"]     = alias_short
    per_phase_df["model_id"]  = model_id
    per_phase_df["precision"] = prec
    per_phase_df["tokens_out"]= [tokens_out, None, None]
    per_phase_df["ppl"]       = [None, ppl, None]
    per_phase_df["bleu"]      = [None, None, bleu]

    # Abgeleitete Kennzahlen
    per_phase_df["wh_total"] = per_phase_df["energy_kwh"] * 1000.0
    per_phase_df["tokens_s"] = None
    per_phase_df.loc[per_phase_df["phase"]=="gen", "tokens_s"] = (
        per_phase_df.loc[per_phase_df["phase"]=="gen", "tokens_out"]
        / per_phase_df.loc[per_phase_df["phase"]=="gen", "time_s"]
    )
    per_phase_df["wh_per_token"] = None
    per_phase_df.loc[per_phase_df["phase"]=="gen", "wh_per_token"] = (
        per_phase_df.loc[per_phase_df["phase"]=="gen", "wh_total"]
        / per_phase_df.loc[per_phase_df["phase"]=="gen", "tokens_out"]
    )
    per_phase_df["s_per_example"] = None
    per_phase_df.loc[per_phase_df["phase"]=="bleu", "s_per_example"] = (
        per_phase_df.loc[per_phase_df["phase"]=="bleu", "time_s"] / float(BLEU_N)
    )
    per_phase_df["wh_per_example"] = None
    per_phase_df.loc[per_phase_df["phase"]=="bleu", "wh_per_example"] = (
        per_phase_df.loc[per_phase_df["phase"]=="bleu", "wh_total"] / float(BLEU_N)
    )
    per_phase_df["kg_per_kwh"] = (per_phase_df["co2_kg"] / per_phase_df["energy_kwh"]).replace([np.inf, -np.inf], np.nan)

    print(f"\nPer-Phase ({alias_short}) — Standort:",
          f"GCP {CLOUD_REGION}" if USE_GCP_REGION else f"Ländermix {COUNTRY_ISO_CODE}")
    print(per_phase_df[[
        "phase","time_s","energy_kwh","co2_kg","kg_per_kwh",
        "tokens_out","ppl","bleu","tokens_s","wh_per_token","s_per_example","wh_per_example"
    ]])

    res = BaselineResult(
        model_id=model_id, alias=alias_short, precision=prec,
        time_s=total_time, energy_kwh=total_energy, co2_kg=total_co2,
        tokens_out=int(tokens_out),
        ram_GB=bytes_to_gb(ram), vram_alloc_GB=bytes_to_gb(valloc), vram_reserved_GB=bytes_to_gb(vres),
        ppl=ppl, bleu=bleu, notes=f"GPU={gpu_name}, VRAM={vram_total_gb:.1f} GB"
    )
    return res, examples, per_phase_df

In [ ]:
# --- Ausführen & Speichern ---
print(f"\n### Starte Baseline (per Phase): {ALIAS_LONG}")
res, examples, phase_df = run_baseline(MODEL_ID, ALIAS_LONG, ALIAS)

df = pd.DataFrame([asdict(res)])
df["kg_per_kwh"] = (df["co2_kg"] / df["energy_kwh"]).replace([np.inf, -np.inf], np.nan)

print("\nGesamt (summiert über Phasen):")
print(df[["model_id","alias","precision","time_s","energy_kwh","co2_kg","kg_per_kwh",
          "tokens_out","ppl","bleu","ram_GB","vram_alloc_GB","vram_reserved_GB","notes"]])

out_dir = project_path
df.to_csv(os.path.join(out_dir, "baseline_deepseek_results.csv"), index=False)
phase_df.to_csv(os.path.join(out_dir, "baseline_deepseek_per_phase.csv"), index=False)
print("Gespeichert (gesamt):", os.path.join(out_dir, "baseline_deepseek_results.csv"))
print("Gespeichert (per Phase):", os.path.join(out_dir, "baseline_deepseek_per_phase.csv"))

samples_path = os.path.join(out_dir, f"baseline_samples_{ALIAS}.txt")
with open(samples_path, "w", encoding="utf-8") as f:
    for i, txt in enumerate(examples, 1):
        f.write(f"--- Beispiel {i} ({ALIAS}) ---\n{txt}\n\n")
print("Beispiele gespeichert:", samples_path)

print("\nEmissions-Logs (pro Phase):")
print(f" - deepseek_{ALIAS}_gen.csv")
print(f" - deepseek_{ALIAS}_ppl.csv")
print(f" - deepseek_{ALIAS}_bleu.csv")


### Starte Baseline (per Phase): DeepSeek-R1-Distill-Qwen-1.5B


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

de-en/test-00000-of-00001.parquet:   0%|          | 0.00/509k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]


Per-Phase (r1q15b) — Standort: GCP europe-west10
  phase     time_s  energy_kwh    co2_kg  kg_per_kwh  tokens_out        ppl  \
0   gen   3.172588    0.000094  0.000043    0.452621        96.0        NaN   
1   ppl   3.948225    0.000108  0.000049    0.452621         NaN  212.92872   
2  bleu  43.889363    0.001268  0.000574    0.452621         NaN        NaN   

        bleu   tokens_s wh_per_token s_per_example wh_per_example  
0        NaN  30.259207     0.000983          None           None  
1        NaN       None         None          None           None  
2  12.974741       None         None      1.371543       0.039618  

Gesamt (summiert über Phasen):
                                    model_id   alias precision     time_s  \
0  deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B  r1q15b      fp16  51.010176   

   energy_kwh    co2_kg  kg_per_kwh  tokens_out        ppl       bleu  \
0     0.00147  0.000665    0.452621          96  212.92872  12.974741   

     ram_GB  vram_alloc_GB 